In [ ]:
import scipp as sc
import numpy as np
import matplotlib.pyplot as plt
import plopp as pp
%matplotlib widget
from scipp import curve_fit
from scipy.special import voigt_profile



In [ ]:
#TODO Make a general data loader
## Load actual data - just one temperature for now
number_of_Q_points=16

#Preallocate
intensity_values=np.zeros((number_of_Q_points,1024))
error_values=np.zeros((number_of_Q_points,1024))

# Load data into a matrix
for Q in range(16):
    filename = '../../IN16b_GGG_data/data_450mK_Q' +str(Q+1) +'.dat'

    data_array = np.loadtxt(filename)
    energy_values=data_array[:, 0] #should be the same for all Q
    # EnergyValues[Q,:]=data_array[:, 0]
    intensity_values[Q,:]=data_array[:,1]
    error_values[Q,:]=data_array[:,2]

# Define energy, q and intensity as scipp variables with units, and make a DataArraw
energy=sc.array(dims=['energy'],values=energy_values/1000,unit='meV')
Q=sc.array(dims=['Q'],values=range(number_of_Q_points))
intensity=sc.array(dims=['Q','energy'],values=intensity_values,variances=error_values*error_values,unit='dimensionless') #The variance is the square of the uncertainty!

GGG_data_450mK = sc.DataArray(data=intensity, coords={'Q':Q,'energy': energy})

In [14]:

def voigt_problem_ufunc(energy, sigma,lorz1_area,lorz1_HWHM,BG):

    meV=1*sc.Unit('meV')
    y_lorz=lorz1_area/meV*(voigt_profile(energy, sigma, lorz1_HWHM))
    y = y_lorz + BG
    return y

sigma= 4e-4 * sc.Unit('meV')         
lorz1_area      = 5e-5*sc.Unit('meV')
lorz1_HWHM      = 4e-4 * sc.Unit('meV')
BG              = 1e-5 * sc.Unit('dimensionless')

yy=voigt_problem_ufunc(energy, sigma,lorz1_area,lorz1_HWHM,BG)
print(yy)


popt_GGG, _ = curve_fit(['energy'], voigt_problem_ufunc, GGG_data_450mK, 
                    p0 = {
                        'sigma'           : 4e-4 * sc.Unit('meV'),
                        'lorz1_area'      : 5e-5*sc.Unit('meV'), 
                        'lorz1_HWHM'      : 5e-5*sc.Unit('meV'), 
                        'BG'              :1e-5
                        })


TypeError: operand 'scipp._scipp.core.Variable' does not support ufuncs (__array_ufunc__=None)

In [39]:

def voigt_problem(energy, sigma,lorz1_area,lorz1_HWHM,BG):


    y_lorz=lorz1_area.value*(voigt_profile(energy.values, sigma.value, lorz1_HWHM.value))
        
    y = y_lorz + BG.value
    y=sc.array(dims=['energy'],values=y,unit='dimensionless')
    return y

sigma           = 4e-4 * sc.Unit('meV')         
lorz1_area      = 5e-5 *sc.Unit('meV')
lorz1_HWHM      = 4e-4 * sc.Unit('meV')
BG              = 1e-5 * sc.Unit('dimensionless')

yy=voigt_problem(energy, sigma,lorz1_area,lorz1_HWHM,BG)
print(yy)


popt_GGG, _ = curve_fit(['energy'], voigt_problem, GGG_data_450mK, 
                    p0 = {
                        'sigma'           : 4e-4 * sc.Unit('meV'),
                        'lorz1_area'      : 5e-5*sc.Unit('meV'), 
                        'lorz1_HWHM'      : 5e-5*sc.Unit('meV'), 
                        'BG'              : 1e-5 * sc.Unit('dimensionless')
                        })



<scipp.Variable> (energy: 1024)    float64  [dimensionless]  [1.72305e-05, 1.72589e-05, ..., 1.72589e-05, 1.72305e-05]


In [18]:
sigma.value

0.0004

In [40]:
def voigt_problem2(energy, sigma,lorz1_area,lorz1_HWHM,BG):


    y_lorz=lorz1_area.value*(voigt_profile(energy.values, sigma.value, lorz1_HWHM.value))
        
    y = y_lorz + BG.values
    y=sc.array(dims=['energy'],values=y,unit='dimensionless')
    return y

sigma           = 4e-4 * sc.Unit('meV')         
lorz1_area      = 5e-5 *sc.Unit('meV')
lorz1_HWHM      = 4e-4 * sc.Unit('meV')
BG              = 1e-5 * sc.Unit('dimensionless')

yy=voigt_problem2(energy, sigma,lorz1_area,lorz1_HWHM,BG)
print(yy)


popt_GGG, _ = curve_fit(['energy'], voigt_problem, GGG_data_450mK, 
                    p0 = {
                        'sigma'           : 4e-4 * sc.Unit('meV'),
                        'lorz1_area'      : 5e-5*sc.Unit('meV'), 
                        'lorz1_HWHM'      : 5e-5*sc.Unit('meV'), 
                        'BG'              : 1e-5 * sc.Unit('dimensionless')
                        })



<scipp.Variable> (energy: 1024)    float64  [dimensionless]  [1.72305e-05, 1.72589e-05, ..., 1.72589e-05, 1.72305e-05]


In [ ]:
from scipy.special import voigt_profile
# energy=sc.array(dims=['energy'],values=,unit='meV')
energy=sc.linspace('energy', start=-2, stop=2, num=1001, unit='meV')
sigma= 0.1 * sc.Unit('meV')         
lorz1_area      = 5*sc.Unit('meV')
lorz1_HWHM      = 0.1 * sc.Unit('meV')
BG              = 0.1 * sc.Unit('dimensionless')

# voigt_profile(energy,sigma,lorz1_HWHM)





In [ ]:
y=lorz1_area.value*voigt_profile(energy.values, sigma.value, lorz1_HWHM.value) 
plt.clf()
plt.plot(energy.values,y)
plt.show()

In [ ]:
meV=1*sc.Unit('meV')
y=voigt_profile(energy.values, sigma.value, lorz1_HWHM.value)
y_lorz=lorz1_area/meV*sc.Variable(dims=['energy'],values=y)

sc.plot(y_lorz)

In [ ]:
import numpy as np
from scipy.special import wofz

lorz1_HWHM = 5e-5*sc.Unit('meV')
sigma = 4e-4 * sc.Unit('meV')
x=energy

def voigt(x, sigma, gamma):
    z = (x + 1j * gamma) / (sigma * np.sqrt(2))
    return np.real(wofz(z).real)


y_values = voigt(energy, sigma, lorz1_HWHM)

In [30]:
# sc.show(GGG_data_450mK)
GGG_data_450mK

<scipp.DataArray>
Dimensions: Sizes[Q:16, energy:1024, ]
Coordinates:
* Q                           int32  [dimensionless]  (Q)  [0, 1, ..., 14, 15]
* energy                    float64            [meV]  (energy)  [-0.029678, -0.02962, ..., 0.02962, 0.029678]
Data:
                            float64  [dimensionless]  (Q, energy)  [0, 0, ..., 0, 0]  [1, 1, ..., 1, 1]

In [31]:
energy

<scipp.Variable> (energy: 1024)    float64            [meV]  [-0.029678, -0.02962, ..., 0.02962, 0.029678]

In [36]:
import scipp.compat.wrapping
@sc.compat.wrapping.wrap1d()
def voigt_problem_wrap(energy, sigma,lorz1_area,lorz1_HWHM,BG):


    y_lorz=lorz1_area.value*(voigt_profile(energy.values, sigma.value, lorz1_HWHM.value))
        
    y = y_lorz + BG.value
    # y=sc.array(dims=['energy'],values=y,unit='dimensionless')
    return y

sigma           = 4e-4 * sc.Unit('meV')         
lorz1_area      = 5e-5 *sc.Unit('meV')
lorz1_HWHM      = 4e-4 * sc.Unit('meV')
BG              = 1e-5 * sc.Unit('dimensionless')

yy=voigt_problem_wrap(energy, sigma,lorz1_area,lorz1_HWHM,BG)
print(yy)


popt_GGG, _ = curve_fit(['energy'], voigt_problem_wrap, GGG_data_450mK, 
                    p0 = {
                        'sigma'           : 4e-4 * sc.Unit('meV'),
                        'lorz1_area'      : 5e-5*sc.Unit('meV'), 
                        'lorz1_HWHM'      : 5e-5*sc.Unit('meV'), 
                        'BG'              : 1e-5 * sc.Unit('dimensionless')
                        })

TypeError: voigt_problem_wrap() takes 2 positional arguments but 5 were given